# Welcome to the wrf-python Tutorial!

# Jupyter Notebook Quick Reference

**To execute a cell in jupyter notebook**:

1. Click on the desired cell.
2. Press **CTRL + RETURN** to execute the cell or press **SHIFT + RETURN** to execute the cell and advance to the next cell.
3. Alternatively, you can use the Cell dropdown menu

**If you accidentally double click on a Markdown Cell (cells with no code, only pretty text)**.

You will usually see text, and most likely blue '#' characters and letters
1. Simply execute the cell using the instructions above

**If you experience a problem and want to restart your notebook**:

1. Use the Kernel dropdown menu at the top.
2. Execute Kernel -> Restart & Clear Output.
3. Be sure the run **Example 1.1: Verifying your Jupyter Environment** before running any 
   other cells.


**Shutting down the notebook**

1. On your web browser, select the Home tab.
2. Click the check box next to wrf_python_student.ipynb.
3. Click the Shutdown button that will become available after step 2.
4. Now go to the terminal window where you typed in "jupyter notebook".
5. With the terminal window active, press **CTRL + C**.


# 1.0 Introduction to jupyter, numpy, xarray

## Example 1.1 Verifying your Jupyter Environment

Looking at your own WRF data is much more fun than my examples!  If you have your own data files:

1. Place the WRF output files in to a single directory.  Pick two or three files to limit memory.
2. In the cell below, modify the **WRF_DIRECTORY** and **WRF_FILES** variables to point to your data.
3. Execute the cell and verify that "All Tests Passed!" is printed.
4. If not, raise your hand and one of the lab assistants will help.

**IMPORTANT: If for some reason your workbook crashes, you need to run this cell again before running the later examples**.

In [ ]:
from __future__ import print_function

# This jupyter notebook command inserts matplotlib graphics in 
# to the workbook
%matplotlib inline

# Modify these to point to your own files
WRF_DIRECTORY = "./"
WRF_FILES = ["wrfout_d01_2005-08-28_00:00:00",
             "wrfout_d01_2005-08-28_12:00:00"]


# Do not modify the code below this line
#------------------------------------------------------
# Make sure the environment is good
import numpy
import cartopy
import matplotlib
from netCDF4 import Dataset
from xarray import DataArray
from wrf import (getvar, interplevel, vertcross, 
                 vinterp, ALL_TIMES)
import os

_WRF_FILES = [os.path.abspath(os.path.expanduser(
    os.path.join(WRF_DIRECTORY, f))) for f in WRF_FILES]

# Check that the WRF files exist
for f in _WRF_FILES:
    if not os.path.exists(f):
        raise ValueError("{} does not exist. "
            "Check for typos or incorrect directory.".format(full_path))

# Create functions so that the WRF files only need
# to be specified using the WRF_FILES global above
def single_wrf_file():
    global _WRF_FILES
    return _WRF_FILES[0]

def multiple_wrf_files():
    global _WRF_FILES
    return _WRF_FILES

print ("All tests passed!")
    


# Numpy Way-Too-Quick Reference

See https://docs.scipy.org/doc/numpy/user/quickstart.html for a much more complete introduction.

## Creating an Array

The zeros function creates an array with all 0's.  

Begin by specifying the desired shape as a tuple and then specify the data type ("float32" is the default if not specified).

``` python
my_array = numpy.zeros((3,3,3), "float32")
```

## Accessing Elements

Use the "[ ]" syntax with the desired dimension indexes separated by commas.

``` python

first_element = my_array[0,0,0]

last_element = my_array[-1,-1,-1]

mid_element = my_array[1,1,1]

# Setting an element
my_array[1,1,1] = 10.0

```

## Slices

Use the 'start : end' syntax for slicing.  

If *start* is left blank, the slice begins at the start of the array.  

If *end* is blank, the slice ends at the end of the array.

``` python
import numpy

my_array = numpy.zeros((3,3,3), "float32")

first_row = my_array[0,0,:]

first_column = my_array[0,:,0]

first_z = my_array[:,0,0]

subset = my_array[:, :, 1:3]

```

## MaskedArray

Use the *masked_{condition}* functions to create MaskedArray objects.

``` python

import numpy
import numpy.ma

my_array = numpy.zeros((3,3,3), "float32")

# Now all of the array elements are masked values
my_masked = numpy.ma.masked_equal(my_array, 0)

```


## Example 1.2: Numpy Basics

This example demonstrates the basics of creating a numpy array, getting and setting an array element, slicing, and masking values.

In [ ]:
import numpy
import numpy.ma

my_array = numpy.zeros((3,3,3), "float32")

print ("my_array")
print (my_array)
print ("\n")

# Setting an element
my_array[1,1,1] = 10.0

# Getting an element
mid = my_array[1,1,1]

print ("Mid element set")
print (my_array)
print ("\n")

# Getting a slice
my_slice = my_array[1,:,:]

print ("my_slice")
print (my_slice)
print ("\n")

# Masking the zeros
my_masked = numpy.ma.masked_equal(my_array, 0)

print ("my_masked")
print (my_masked)
print ("\n")

# xarray Basic Way-Too-Quick Reference

See http://xarray.pydata.org/ for a much more complete reference on xarray.

## Creating an xarray DataArray


Arguments for creating a DataArray are:

- data [required]: A numpy array.
- coords [optional]: A dictionary of {dimension_name : coordinate_array}.
- dims [optional]: A list of dimension names from left to right.
- name [optional]: A name for the variable.
- attrs[optional]: A dictionary of {attr_name : attr_value}.

``` python

import numpy
import xarray

my_array = numpy.zeros((3,3,3), "float32")

# Making up dimension names and coordinates.
my_name = "my_xarray"

my_dims = ["bottom_top", "south_north", "west_east"]

my_coords = {"bottom_top" : [100., 200., 300.],
          "south_north": [40., 50., 60.],
          "west_east" : [-120., -110., -100.]
         }

my_attrs = {"info" : "This is my xarray"}

my_xarray = xarray.DataArray(my_array,
                           name=my_name,
                           dims=my_dims, 
                           coords=my_coords, 
                           attrs=my_attrs)

```

## xarray Arrays with Missing Values

- xarray always uses NaN for missing values.
- This can cause problems with compiled math rouines.
- The xarray.DataArray contains a .values property to get the numpy array, but this will not 
  replace the NaN values with the _FillValue values (see fillna and to_masked_array in xarray documentation).
- wrf-python has a *to_np* function that will handle the conversion to numpy for you and fill in the missing values if _FillValue is in the attributes.

## xarray Notes:

- xarray DataArrays support most of the numpy methods and attributes, but not all.
- xarray DataArrays are NOT numpy subclasses.  
- Often you need to extract the numpy array from the DataArray before passing the array to 
  a computational routine.
- However, the routines in wrf-python are xarray-aware and will do this for you.

## Example 1.3: Creating an xarray DataArray

Run the cell below to see what an xarray DataArray looks like.

In [ ]:
import numpy
import xarray

my_array = numpy.zeros((3,3,3), "float32")

# Making up dimension names and coordinates.
my_name = "my_xarray"

my_dims = ["bottom_top", "south_north", "west_east"]

my_coords = {"bottom_top" : [100., 200., 300.],
          "south_north": [40., 50., 60.],
          "west_east" : [-120., -110., -100.]
         }

my_attrs = {"info" : "This is my xarray"}

my_xarray = xarray.DataArray(my_array,
                           name=my_name,
                           dims=my_dims, 
                           coords=my_coords, 
                           attrs=my_attrs)

print (my_xarray)

## Example 1.4: xarray and Missing Values

In this example, an xarray.DataArray is constructed from a MaskedArray.  

Note how the missing values are all NaN.

At the end, the wrf-python *to_np* routine is used to convert the DataArray back to a MaskedArray.

In [ ]:
import numpy
import numpy.ma
import xarray

from wrf import to_np

# Create a MaskedArray with 10.0 in the center
my_array = numpy.zeros((3,3,3), "float32")

my_array[1,1,1] = 10.0

my_masked = numpy.ma.masked_equal(my_array, 0)

# Making up dimension names and 
# coordinates.
my_name = "my_masked_xarray"

my_dims = ["bottom_top", "south_north", "west_east"]

my_coords = {"bottom_top" : [100., 200., 300.],
          "south_north": [40., 50., 60.],
          "west_east" : [-120., -110., -100.]
         }

my_attrs = {"info" : "This is my masked xarray",
           "_FillValue" : -999.0}

# Create the xarray DataArray
my_xarray = xarray.DataArray(my_masked,
                           name=my_name,
                           dims=my_dims, 
                           coords=my_coords, 
                           attrs=my_attrs)

print ("xarray Array with Missing Values")
print (my_xarray)
print ("\n")

# Covert back to a MaskedArray
converted = to_np(my_xarray)

print ("Converted to a MaskedArray with to_np")
print (converted)

# 2.0 Overview of WRF Data

## Example 2.1: Running ncdump

In this example, the ncdump command is called from inside of python.  

Examine the output for your data. 

In [ ]:
from subprocess import Popen, PIPE, STDOUT

file_path = single_wrf_file()

# This simply executes 'ncdump -h {wrf_file}' 
# from Python
p = Popen(["ncdump", "-h", "{}".format(file_path)], 
          stdout=PIPE, stderr=STDOUT)
output, _ = p.communicate()

print (output)

## Example 2.2: Using netcdf4-python

In this example, the netcdf4-python package is used to read your WRF NetCDF file.  


In [ ]:
from netCDF4 import Dataset

file_path = single_wrf_file()

wrf_file = Dataset(file_path)

print(wrf_file)

## Example 2.3: Variables, Attributes, and Data with netcd4-python

In this example, netcdf4-python is used to get the global attributes, get a variable (P), get the variable attributes (coordinates), and get the variable data.

In [ ]:
from netCDF4 import Dataset

file_path = single_wrf_file()

# Create the netCDF4.Dataset object
wrf_file = Dataset(file_path)

# Get the global attribute dict
global_attrs = wrf_file.__dict__
print ("Global attributes for the file")
print(global_attrs)
print ("\n")

# Just get the 'MAP_PROJ' attribute
map_proj = wrf_file.getncattr("MAP_PROJ")
print ("The MAP_PROJ attribute:")
print (map_proj)
print("\n")

# Get the perturbation pressure variable
p = wrf_file.variables["P"]
print ("The P variable: ")
print(p)
print ("\n")

# Get the P attributes
p_attrs = p.__dict__
print ("The attribute dict for P")
print (p_attrs)
print ("\n")

# Get the 'coordinates' attribute for P
coords = p.getncattr("coordinates")
print ("Coordinates for P:")
print (coords)
print ("\n")

# Get the P numpy array for all times
p_all_data = p[:]
print ("The P numpy array: ")
print (p_all_data)
print ("\n")

# Get the P numpy array for time 0
p_t0_data = p[0,:]
print ("P array at time 0:")
print (p_t0_data)
print ("\n")

# 3.0 wrf-python

## Example 3.1: Using *getvar* to Extract a WRF NetCDF Variable

In this example, the *getvar* function is used to read a variable in a WRF NetCDF file.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar

file_path = single_wrf_file()

wrf_file = Dataset(file_path)

hgt = getvar(wrf_file, "HGT", timeidx=0)

print(hgt)

## Example 3.2: Using *getvar* to compte Sea Level Pressure (SLP)

In this example, *getvar* is used to compute a diagnostic variable.

The full table of avaiable diagnostics is here:  http://wrf-python.readthedocs.io/en/latest/diagnostics.html

Also try changing the units for by specifiying the following values: 'hPa', 'Pa', 'atm', 'mmhg'

In [ ]:
from netCDF4 import Dataset
from wrf import getvar

file_path = single_wrf_file()

wrf_file = Dataset(file_path)

slp = getvar(wrf_file, "slp", timeidx=0, units="mmhg")

print (slp)

## Example 3.3: Combining Files Using the 'cat' Method

In this example, the variable is computed for all times, across all files, and combined along the Time dimension by using the 'cat' method

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = multiple_wrf_files()

wrf_files = [Dataset(file_paths[0]),
             Dataset(file_paths[1])]

slp = getvar(wrf_files, "slp", timeidx=ALL_TIMES, method="cat")

print (slp)

## Example 3.4: Combining Files Using the 'join' Method

In this example, the variable is computed for all times, across all files, and combined by creating a new leftmost index for the file by using the 'join' method.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = multiple_wrf_files()

wrf_files = [Dataset(file_paths[0]),
             Dataset(file_paths[1])]

slp = getvar(wrf_files, "slp", timeidx=ALL_TIMES, method="join")

print (slp)

## Example 3.5: Interpolate to 500 hPa Using *interplevel*

In this example, the 500 hPa geopotential heights are calculated by using the *interplevel* function.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, interplevel

file_path = single_wrf_file()

wrf_file = Dataset(file_path)

pres = getvar(wrf_file, "pressure", timeidx=0)
ht = getvar(wrf_file, "z", timeidx=0, units="dm")

ht_500 = interplevel(ht, pres, 500.0)

print (ht_500)

## Example 3.6: Interpolate to a Vertical Cross Section with *vertcross*

In this example, the vertical cross section for wind speed is calculated using the *vertcross* function.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, vertcross, CoordPair

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Making a diagonal cross section line from 
# bottom left to top right.
bottom_left = CoordPair(x=0, y=0)

top_right = CoordPair(x=-1, y=-1)

# Let's get wind speed in kts
wspd_wdir = getvar(wrf_file, "wspd_wdir", 
                   timeidx=0, units="kt")
                   
wspd = wspd_wdir[0,:]

# Get the height levels
ht = getvar(wrf_file, "z", timeidx=0)

wspd_cross = vertcross(wspd, ht, 
               start_point=bottom_left, 
               end_point=top_right)

print (wspd_cross)

## Example 3.7: Interpolate to Theta-e Levels with *vinterp*

In this example, pressure is interpolated to various theta-e levels by using the *vinterp* function.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, vinterp 

file_path = "./wrfout_d01_2005-08-28_00:00:00"
wrf_file = Dataset(file_path) 

pres = getvar(wrf_file, "pressure", timeidx=0)

# Interpolate pressure to theta-e levels                 
interp_levels = [280., 285., 290., 292., 294., 
                 296., 298., 300., 305., 310.]

pres_eth = vinterp(wrf_file, 
               field=pres, 
               vert_coord="theta-e", 
               interp_levels=interp_levels, 
               extrapolate=True, 
               field_type="pressure", 
               log_p=False,
               timeidx=0)

print (pres_eth)

## Example 3.8: *xy_to_ll* and *ll_to_xy*

In this example, several x,y coordinate values are converted to latitude,longitude values.  

These latitude,longitude values are then converted back to x,y values.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, xy_to_ll, ll_to_xy 

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

lat_lon = xy_to_ll(wrf_file, [20, 30], [50,75])

print ("lat,lon values")
print (lat_lon)
print ("\n")

x_y = ll_to_xy(wrf_file, lat_lon[0,:], lat_lon[1,:])

print ("x,y values")
print (x_y)


# 4.0 Plotting

## Example 4.1: Single Wind Barb Example with pyplot

In this example, a single wind barb is created in the center of the domain (by masking all other values).

This simple example uses only the matplotlib.pyplot API.

In [ ]:
from matplotlib import pyplot
import numpy as np

# Make a 5x5 grid of missing u,v values
u = np.ma.masked_equal(np.zeros((5,5)), 0)
v = np.ma.masked_equal(np.zeros((5,5)), 0)

# Add u,v winds to center of domain
u[2,2] = 10.0
v[2,2] = 10.0

# Draw a single wind barb in the middle using pyplot API
# Note:  the axes objects are "hidden" in these functions
fig = pyplot.figure()
pyplot.barbs(u, v)

# Set the x and y ranges so the barb is in the middle
pyplot.xlim(0, 4)
pyplot.ylim(0, 4)

pyplot.show()

## Example 4.2: Single Wind Barb Using the Axes Object

This is the same example as before, but now the Axes objects are used directly, rather than the pure pyplot API.

Often you'll use a mix of pyplot and the object API when working with matplotlib, especially when making panel plots.

In [ ]:
from matplotlib import pyplot
import numpy as np

# Make a 5x5 grid of missing u,v values
u = np.ma.masked_equal(np.zeros((5,5)), 0)
v = np.ma.masked_equal(np.zeros((5,5)), 0)

# Add u,v winds to center of domain
u[2,2] = 10.0
v[2,2] = 10.0

# We'll use pyplot to create the figure and 
# get the axes
fig = pyplot.figure()
ax = pyplot.axes() # <- Remember this line

# Now use the axes directly to create the barbs
ax.barbs(u, v)

# Set the x and y ranges using the axes directly
ax.set_xlim(0, 4)
ax.set_ylim(0, 4)

pyplot.show()

## Example 4.3: Making a Plot of Terrain

In this example, terrain is plotted.  Plotting terrain is a good way of checking your data is mapped correctly.

In [ ]:
import numpy
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the terrain height
terrain = getvar(wrf_file, "ter", timeidx=0)

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(terrain)
lats, lons = latlon_coords(terrain)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(10, 7.5))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=.5)
geo_axes.coastlines('50m', linewidth=0.8)

# Set the contour levels
levels = numpy.arange(250., 5000., 250.)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(terrain), levels=levels, 
               colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(terrain), levels=levels,
                transform=crs.PlateCarree(),
                cmap=get_cmap("terrain"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=.86)

pyplot.show()

## Example 4.4: Plot of Sea Level Pressure

In this example, sea level pressure is plotted.

The following examples show how to crop this figure.

In [ ]:
import numpy
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the terrain height
slp = getvar(wrf_file, "slp", timeidx=0)

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(slp)
lats, lons = latlon_coords(slp)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(10, 7.5))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=.5)
geo_axes.coastlines('50m', linewidth=0.8)

# Set the contour levels so that all plots match
levels = numpy.arange(980.,1030.,2.5)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(slp), levels=levels, colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(slp), levels=levels, 
                transform=crs.PlateCarree(),
                cmap=get_cmap("jet"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=.86)

pyplot.show()

## Example 4.5: Cropping by Slicing the Data

In this example, the data is cropped to the lower right quadrant using slicing before plotting.

In [ ]:
import numpy
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the terrain height
slp = getvar(wrf_file, "slp", timeidx=0)

# Determine the center of the domain in grid coordinates
slp_shape = slp.shape
center_y = int(slp_shape[-2]/2.) - 1
center_x = int(slp_shape[-1]/2.) - 1

# Slice from bottom to middle for y
# Slice from middle to right for x
slp_quad = slp[..., 0:center_y+1, center_x:]

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(slp_quad)
lats, lons = latlon_coords(slp_quad)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(10, 7.5))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=.5)
geo_axes.coastlines('50m', linewidth=0.8)

# Set the contour levels so that all plots match
levels = numpy.arange(980.,1030.,2.5)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(slp_quad), levels=levels, colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(slp_quad), levels=levels, 
                transform=crs.PlateCarree(),
                cmap=get_cmap("jet"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=.83)

pyplot.show()

## Example 4.6: Cropping by Setting the X,Y Extents

In this example, the plot is cropped to the lower right quardrant by setting x and y axis extents.

In [ ]:
import numpy
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords
from wrf import xy_to_ll, cartopy_xlim, cartopy_ylim
from wrf import CoordPair, GeoBounds

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the terrain height
slp = getvar(wrf_file, "slp", timeidx=0)

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(slp)
lats, lons = latlon_coords(slp)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(10, 7.5))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=.5)
geo_axes.coastlines('50m', linewidth=0.8)

# Set the contour levels so that all plots match
levels = numpy.arange(980.,1030.,2.5)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(slp), levels=levels, colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(slp), levels=levels, 
                transform=crs.PlateCarree(),
                cmap=get_cmap("jet"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=.83)

# Set up the x, y extents

# Determine the center of the domain in grid coordinates
slp_shape = slp.shape
start_y = 0
center_y = int(slp_shape[-2]/2.) - 1
center_x = int(slp_shape[-1]/2.) - 1
end_x = int(slp_shape[-1]) - 1

# Get the lats and lons for the start, center, and end points
# (Normally you would just set these yourself)
center_latlon = xy_to_ll(wrf_file, 
                         [center_x, end_x], 
                         [start_y, center_y])

start_lat = center_latlon[0,0]
end_lat = center_latlon[0,1]
start_lon = center_latlon[1,0]
end_lon = center_latlon[1,1]

# Set the extents
geo_bounds = GeoBounds(CoordPair(lat=start_lat, lon=start_lon),
                       CoordPair(lat=end_lat, lon=end_lon))
geo_axes.set_xlim(cartopy_xlim(slp, geobounds=geo_bounds))
geo_axes.set_ylim(cartopy_ylim(slp, geobounds=geo_bounds))

pyplot.show()